In [2]:
import os
os.chdir('..')

In [4]:
from src import CustomGraph, AddData, Wrapper

In [5]:
from src.utils import list_from_file

## documentation

In [6]:
g = CustomGraph()

In [8]:
g.parse("http://www.semanticweb.org/football/", "./KG/data/football_antology.rdf")

In [11]:
# add classes, objects etc
file_path = "./KG/data/data_lists/"

In [12]:
classes = list_from_file(file_path+"classes")
obj_prop = list_from_file(file_path+"obj_prop")
data_prop = list_from_file(file_path+"data_prop")

In [13]:
for clss in classes:
    g.add_class(clss)
for opr in obj_prop:
    g.add_object_property(opr)
for dpr in data_prop:
    g.add_data_property(dpr)

In [14]:
# fullfill the antology
wrapper = Wrapper()

In [18]:
generator = wrapper.set_query(
"""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?stadiumName ?cityName ?countryName WHERE {
  ?stadium rdf:type dbo:Stadium .
  ?stadium dbo:location ?city .
  ?city dbo:country ?country .
  ?stadium rdfs:label ?stadiumName .
  ?city rdfs:label ?cityName .
  ?country rdfs:label ?countryName .
  FILTER (LANG(?stadiumName) = 'en')
  FILTER (LANG(?cityName) = 'en')
  FILTER (LANG(?countryName) = 'en')
}
LIMIT 10000

"""
)

In [17]:
for i in generator:
    print(i)

In [20]:
# AddData.add_stadium(
#     g: CustomGraph, 
#     country_name: object, 
#     city_name: object, 
#     stadium_name: object
# )

generator = wrapper.set_query(
"""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?stadiumName ?cityName ?countryName WHERE {
  ?stadium rdf:type dbo:Stadium .
  ?stadium dbo:location ?city .
  ?city dbo:country ?country .
  ?stadium rdfs:label ?stadiumName .
  ?city rdfs:label ?cityName .
  ?country rdfs:label ?countryName .
  FILTER (LANG(?stadiumName) = 'en')
  FILTER (LANG(?cityName) = 'en')
  FILTER (LANG(?countryName) = 'en')
}
LIMIT 10000

"""
)
for i in generator:
    print(i)

{'stadiumName': 'Roberto_Clemente_Stadium', 'cityName': 'Carolina,_Puerto_Rico', 'countryName': 'Puerto_Rico'}
{'stadiumName': 'Prienai_Arena', 'cityName': 'Prienai', 'countryName': 'Lithuania'}
{'stadiumName': 'Páirc_Seán_Mac_Diarmada', 'cityName': 'Carrick-on-Shannon', 'countryName': 'Republic_of_Ireland'}
{'stadiumName': 'Cardiff_City_Stadium', 'cityName': 'Cardiff', 'countryName': 'United_Kingdom'}
{'stadiumName': 'Carrington_Training_Centre', 'cityName': 'Carrington_Moss', 'countryName': 'United_Kingdom'}
{'stadiumName': 'Rochester_Stadium', 'cityName': 'Rochester,_Kent', 'countryName': 'United_Kingdom'}
{'stadiumName': 'Cardiff_International_Sports_Stadium', 'cityName': 'Cardiff', 'countryName': 'United_Kingdom'}
{'stadiumName': 'Cardiff_Athletics_Stadium', 'cityName': 'Cardiff', 'countryName': 'United_Kingdom'}
{'stadiumName': 'Carrington_Training_Centre', 'cityName': 'Carrington,_Greater_Manchester', 'countryName': 'United_Kingdom'}
{'stadiumName': 'QU_Stadium', 'cityName': 'Qu

In [34]:
# AddData.add_competition(
#     g: CustomGraph,
#     competition_type: object,
#     competition_name: object,
#     competition_start_date: Optional[object],
#     compettiton_end_date: Optional[object],
#     competition_statistic: Optional[object],
# )


from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""
SELECT ?competition ?competitionLabel ?startDate ?endDate
WHERE {
  ?competition wdt:P31 wd:Q27020041; # wd:Q27020041 is the Wikidata item for 'sports season'.
             wdt:P641 wd:Q2736;     # wd:Q2736 is the Wikidata item for 'association football'.
             wdt:P17 wd:Q46;        # Placeholder for 'Europe'.
             wdt:P580 ?startDate.
  OPTIONAL { ?competition wdt:P582 ?endDate. }
  FILTER(YEAR(?startDate) >= 2010)
  FILTER(!BOUND(?endDate) || YEAR(?endDate) <= 2023)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 100


""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(f"{result['competitionLabel']['value']} - Start: {result['startDate']['value']} End: {result['endDate']['value']}")


In [33]:
# AddData.add_team(
#     g: CustomGraph,
#     team_type: object,
#     team_name: object,
#     team_found_date: Optional[object],
#     team_close_date: Optional[object],
#     team_statistic: Optional[object],
# )
# --------------------
# from SPARQLWrapper import SPARQLWrapper, JSON

# # Setting up the SPARQL endpoint and the query
# sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
# sparql.setQuery("""
# SELECT ?team ?teamName ?sport ?sportName ?city ?cityName
# WHERE {
#   ?team wdt:P31 wd:Q12973014; # Assuming Q12973014 is the Wikidata item for 'sports team'.
#         wdt:P641 ?sport;      # P641 is the property for 'sport'.
#         wdt:P159 ?city;       # P159 is the property for 'headquarters location', often used for the team's city.
#         rdfs:label ?teamName.
#   ?sport rdfs:label ?sportName.
#   ?city rdfs:label ?cityName.
#   FILTER(LANG(?teamName) = "en").
#   FILTER(LANG(?sportName) = "en").
#   FILTER(LANG(?cityName) = "en").
# }
# LIMIT 100
# """)
# sparql.setReturnFormat(JSON)

# # Executing the query and printing results
# try:
#     results = sparql.query().convert()

#     for result in results["results"]["bindings"]:
#         print(f"Team: {result['teamName']['value']}, Sport: {result['sportName']['value']}, City: {result['cityName']['value']}")
# except Exception as e:
#     error_message = str(e)

# error_message if 'error_message' in locals() else 'Query executed successfully'

# --------------------
from SPARQLWrapper import SPARQLWrapper, JSON

# Setting up the SPARQL endpoint and the query
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?team ?teamName ?foundingDate
WHERE {
  ?team wdt:P31 wd:Q476028; # Ensure this is the correct item for 'football team'.
        wdt:P641 wd:Q2736;   # wd:Q2736 is for 'association football' (soccer).
        rdfs:label ?teamName;
        wdt:P571 ?foundingDate.
  FILTER(LANG(?teamName) = "en").
}
LIMIT 100
""")
sparql.setReturnFormat(JSON)

# Executing the query and printing results
try:
    results = sparql.query().convert()

    for result in results["results"]["bindings"]:
        team_name = result['teamName']['value']
        founding_date = result['foundingDate']['value'] if 'foundingDate' in result else 'Not available'
        print(f"Team: {team_name}, Founding Date: {founding_date}")
except Exception as e:
    error_message = str(e)

error_message if 'error_message' in locals() else 'Query executed successfully'



Team: Wrexham A.F.C., Founding Date: 1864-01-01T00:00:00Z
Team: FC Augsburg, Founding Date: 1907-01-01T00:00:00Z
Team: TSV Straubing, Founding Date: 1921-01-01T00:00:00Z
Team: 1. FC Nürnberg, Founding Date: 1900-01-01T00:00:00Z
Team: FC Bayern Munich, Founding Date: 1900-02-27T00:00:00Z
Team: A.S. Casale Calcio, Founding Date: 1909-01-01T00:00:00Z
Team: U.S.D. Novese, Founding Date: 1919-01-01T00:00:00Z
Team: A.C. Perugia Calcio, Founding Date: 1905-01-01T00:00:00Z
Team: Juventus FC, Founding Date: 1897-11-01T00:00:00Z
Team: U.C. Sampdoria, Founding Date: 1946-01-01T00:00:00Z
Team: Club Atlético Osasuna, Founding Date: 1920-10-24T00:00:00Z
Team: Rayo Vallecano, Founding Date: 1924-01-01T00:00:00Z
Team: Rayo Vallecano, Founding Date: 1924-05-29T00:00:00Z
Team: Inter Milan, Founding Date: 1908-03-09T00:00:00Z
Team: U.S. Alessandria Calcio 1912, Founding Date: 1912-01-01T00:00:00Z
Team: Kashima Antlers, Founding Date: 1947-01-01T00:00:00Z
Team: Clube de Regatas do Flamengo, Founding Date:

'Query executed successfully'

In [ ]:
AddData.add_player_position(
    g: CustomGraph, 
    position_name: Optional[object]
)

In [ ]:
AddData.add_player(
    g: CustomGraph,
    player_name: object,
    team_name: object,
    position_name: Optional[object],
    player_emploing_date: Optional[object],
    player_fire_date: Optional[object],
    player_statistic: Optional[object],
)

In [ ]:
AddData.add_coach(
    g: CustomGraph,
    coach_name: object,
    team_name: object,
    coach_emploing_date: Optional[object],
    coach_fire_date: Optional[object],
    coach_statistic: Optional[object],
)

In [ ]:
AddData.add_play(
    g: CustomGraph,
    play_name: object,
    competition_name: object,
    team1_name: object,
    team2_name: object,
    play_date: Optional[object],
    play_statistic: Optional[object],
)

In [ ]:
# end
g.serialize(
    destination='data/football_antology_full.rdf',
    format='xml'
)